In [1]:
# Setup directories
from pathlib import Path
basedir = Path().absolute()
libdir = basedir.parent.parent.parent

# Other imports
import pandas as pd
import numpy as np
from datetime import datetime

from ioos_qc.plotting import bokeh_multi_plot

from bokeh import plotting
from bokeh.layouts import gridplot

In [2]:
# Install QC library
#!pip install git+git://github.com/ioos/ioos_qc.git

# # Alternative installation (install specific branch):
# !pip uninstall -y ioos_qc
# !pip install git+git://github.com/ioos/ioos_qc.git@new_configs

# # Alternative installation (run with local updates):
# !pip uninstall -y ioos_qc
# import sys
# sys.path.append(str(libdir))

## Configuration

In [3]:
erddap_server = 'https://ferret.pmel.noaa.gov/pmel/erddap'
dataset_id = 'sd1035_2019'

## Get data from ERDDAP as an xarray object

In [4]:
from erddapy import ERDDAP
e = ERDDAP(
  server=erddap_server,
  protocol='tabledap',
)
e.response = 'csv'
e.dataset_id = dataset_id

In [5]:
ds = e.to_xarray()
ds

<xarray.Dataset>
Dimensions:            (obs: 189492, trajectory: 1)
Coordinates:
  * trajectory         (trajectory) object '1035.0'
    time               (obs) datetime64[ns] ...
    latitude           (obs) float64 ...
    longitude          (obs) float64 ...
Dimensions without coordinates: obs
Data variables:
    rowSize            (trajectory) int32 ...
    TEMP_AIR_MEAN      (obs) float64 ...
    RH_MEAN            (obs) float64 ...
    BARO_PRES_MEAN     (obs) float64 ...
    TEMP_CTD_RBR_MEAN  (obs) float64 ...
    wind_speed         (obs) float64 ...
    wind_dir           (obs) float64 ...
    SAL_SBE37_MEAN     (obs) float64 ...
Attributes:
    area:                      Arctic Ocean
    cdm_data_type:             Trajectory
    cdm_trajectory_variables:  trajectory
    contributor_email:         heather.tabisola@noaa.gov; oar.pmel.data@noaa.gov
    contributor_name:          Heather Tabisola; PMEL Data Integration Group
    contributor_role:          PMEL Mission Manager; PMEL Data Integration Group
    Conventions:               CF-1.6, COARDS, ACDD-1.3
    creator_email:             support@saildrone.com
    creator_name:              Saildrone
    creator_url:               http://saildrone.com/
    data_mode:                 delay-mode
    date_created:              2019-09-22T10:30:36.948664Z
    date_modified:             2020-02-19T23:05:31Z
    description:               Saildrone NetCDF Format
    drone_id:                  1035
    Easternmost_Easting:       -151.2749056
    featureType:               Trajectory
    funding_source:            NOAA/OAR & OMAO
    geospatial_lat_max:        75.3049536
    geospatial_lat_min:        53.8447616
    geospatial_lat_units:      degrees_north
    geospatial_lon_max:        -151.2749056
    geospatial_lon_min:        -168.9577344
    geospatial_lon_units:      degrees_east
    history:                   2020-02-19T23:05:31Z metadata updated by Scien...
    id:                        sd1035_2019_e698_69e3_e309
    infoUrl:                   https://www.pmel.noaa.gov/
    institution:               NOAA/PMEL; NOAA/AFSC; University of Washington
    interval:                  60 s
    keywords:                  acoustic, active, adcp, afsc, air, air_pressur...
    keywords_vocabulary:       GCMD Science Keywords
    license:                   These data are made freely available without r...
    mission:                   NOAA/PMEL 2019 Arctic Saildrone Mission
    naming_authority:          com.saildrone
    netcdf_version:            4.4.1.1
    Northernmost_Northing:     75.3049536
    platform:                  Saildrone
    project:                   NOAA/PMEL 2019 Arctic Saildrone Mission
    qc_manual:                 No QC performed
    sourceUrl:                 (local files)
    Southernmost_Northing:     53.8447616
    standard_name_vocabulary:  CF Standard Name Table v58
    subsetVariables:           trajectory
    summary:                   Six saildrones (sd-1033, sd-1034, sd-1035, sd-...
    time_coverage_duration:    PT1H59M59S
    time_coverage_end:         2019-09-24T13:59:00Z
    time_coverage_resolution:  PT1M
    time_coverage_start:       2019-05-14T22:00:00Z
    title:                     NOAA/PMEL 2019 Arctic Saildrone Mission, drone...
    Westernmost_Easting:       -168.9577344

## Generate a QC configuration for each variable

In [6]:
# Dataset level metadata to drive climatology extraction
min_t = str(ds.time.min().dt.floor("D").dt.strftime("%Y-%m-%d").data)
max_t = str(ds.time.max().dt.ceil("D").dt.strftime("%Y-%m-%d").data)
min_x = float(ds.longitude.min().data)
min_y = float(ds.latitude.min().data)
max_x = float(ds.longitude.max().data)
max_y = float(ds.latitude.max().data)
bbox = [min_x, min_y, max_x, max_y]

In [7]:
time

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.58 µs


In [8]:
# Configure how each variable's config will be generated
default_config = {
    "bbox": bbox,
    "start_time": min_t,
    "end_time": max_t, 
    "tests": {
        "spike_test": {
            "suspect_threshold": "1",
            "fail_threshold": "2"
        },
        "gross_range_test": {
            "suspect_min": "min - std * 2",
            "suspect_max": "max + std / 2",
            "fail_min": "mean / std",
            "fail_max": "mean * std"
        }
    }
}

# For any variable name or standard_name you can define a custom config
custom_config = {
    'air_temperature': {
        "variable": "air"
    },
    'air_pressure': {
        "variable": "pres"
    },
    'relative_humidity': {
        "variable": "rhum"
    },
    'sea_water_temperature': {
        "variable": "temperature"
    },
    'sea_water_practical_salinity': {
        "variable": "salinity"
    },
    'eastward_wind': {
        "variable": "uwnd"
    },
    'northward_wind': {
        "variable": "vwnd"
    }
}

In [9]:
# Generate climatology configs
from ioos_qc.config_creator import CreatorConfig, QcConfigCreator, QcVariableConfig, QC_CONFIG_CREATOR_SCHEMA

creator_config = {
    "datasets": [
        {
            "name": "ocean_atlas",
            "file_path": "../../../resources/ocean_atlas.nc",
            "variables": {
                "o2": "o_an",
                "salinity": "s_an",
                "temperature": "t_an"
            },
            "3d": "depth"
        },
        {
            "name": "narr",
            "file_path": "../../../resources/narr.nc",
            "variables": {
                "air": "air",
                "pres": "slp",
                "rhum": "rhum",
                "uwnd": "uwnd",
                "vwnd": "vwnd"
            }
        }
    ]
}
cc = CreatorConfig(creator_config)
qccc = QcConfigCreator(cc)

In [10]:
# Break down variable by standard name
def not_stddev(v):
    return v and not v.endswith(' SD')

#air_temp_vars = ds.filter_by_attrs(long_name=not_stddev, standard_name='air_temperature')
#pressure_vars = ds.filter_by_attrs(long_name=not_stddev, standard_name='air_pressure')
# humidity_vars = ds.filter_by_attrs(long_name=not_stddev, standard_name='relative_humidity')
# water_temp_vars = ds.filter_by_attrs(long_name=not_stddev, standard_name='sea_water_temperature')
# salinity_vars = ds.filter_by_attrs(long_name=not_stddev, standard_name='sea_water_practical_salinity')
# uwind_vars = ds.filter_by_attrs(long_name=not_stddev, standard_name='eastward_wind')
# vwind_vars = ds.filter_by_attrs(long_name=not_stddev, standard_name='northward_wind')
# all_vars = [air_temp_vars, pressure_vars, humidity_vars, water_temp_vars, salinity_vars, uwind_vars, vwind_vars]
# all_vars

air_temp = ['air_temperature']
pressure = ['air_pressure']
humidity = ['relative_humidity']
water_temp = ['sea_water_temperature']
salt = ['sea_water_practical_salinity']
u = ['eastward_wind']
v = ['northward_wind']

run_tests = air_temp + pressure + humidity + water_temp + salt + u + v
final_config = {}

for v in ds:
    da = ds[v]

    # Don't run tests for unknown variables
    if 'standard_name' not in da.attrs or da.attrs['standard_name'] not in run_tests:
        continue
    
    # The standard names are identical for the mean and the stddev
    # so ignore the stddev version of the variable
    if v.endswith('_STDDEV'):
        continue
        
    config = default_config.copy()
    
    min_t = str(da.time.min().dt.floor("D").dt.strftime("%Y-%m-%d").data)
    max_t = str(da.time.max().dt.ceil("D").dt.strftime("%Y-%m-%d").data)
    min_x = float(da.longitude.min().data)
    min_y = float(da.latitude.min().data)
    max_x = float(da.longitude.max().data)
    max_y = float(da.latitude.max().data)
    bbox = [min_x, min_y, max_x, max_y]

    config["bbox"] = bbox
    config["start_time"] = min_t
    config["end_time"] = max_t
    
    # Allow custom overrides on a variable name basis
    if v in custom_config:
        config.update(custom_config[v])
    
    # Allow custom overrides on a standard_name name basis
    if da.attrs['standard_name'] in custom_config:
        config.update(custom_config[da.attrs['standard_name']])
        
    # Generate the ioos_qc Config object
    qc_var = QcVariableConfig(config)
    qc_config = qccc.create_config(qc_var)
    
    # Strip off the variable that create_config added
    qc_config = list(qc_config.values())[0]
    
    # Add it to the final config
    final_config[v] = qc_config

In [11]:
final_config

{'TEMP_AIR_MEAN': {'qartod': {'spike_test': {'suspect_threshold': 1.0,
    'fail_threshold': 2.0},
   'gross_range_test': {'suspect_span': [-24.103461168403634,
     16.613016843347552],
    'fail_span': [1.064066625950931, 28.353192330091158]}}},
 'RH_MEAN': {'qartod': {'spike_test': {'suspect_threshold': 1.0,
    'fail_threshold': 2.0},
   'gross_range_test': {'suspect_span': [63.329960408143656,
     100.52467809559592],
    'fail_span': [19.32332059092184, 408.9438763722915]}}},
 'BARO_PRES_MEAN': {'qartod': {'spike_test': {'suspect_threshold': 1.0,
    'fail_threshold': 2.0},
   'gross_range_test': {'suspect_span': [999.3113169802207,
     1019.1693147034075],
    'fail_span': [459.021270896595, 2228.7803822705596]}}},
 'TEMP_CTD_RBR_MEAN': {'qartod': {'spike_test': {'suspect_threshold': 1.0,
    'fail_threshold': 2.0},
   'gross_range_test': {'suspect_span': [-9.84610307722739,
     15.192197175099084],
    'fail_span': [1.2560024049304177, 21.249804349139634]}}},
 'SAL_SBE37_MEA

In [12]:
from ioos_qc.config import Config
from ioos_qc.streams import XarrayStream
from ioos_qc.stores import NetcdfStore

c = Config(final_config)
xs = XarrayStream(ds, time='time', lat='latitude', lon='longitude')
qc_results = xs.run(c)

Could not run "qartod.gross_range_test: User span range may not exceed sensor span
Could not run "qartod.gross_range_test: User span range may not exceed sensor span


In [13]:
title = "PLOTS"
plot = bokeh_multi_plot(xs, qc_results, title, ncols=2, sizing_mode='scale_width')

# This is slow.
# plotting.output_file(filename='test')
# plotting.show(plot)